<a href="https://colab.research.google.com/github/sciencebyAJ/OWRD_Training_20240927/blob/main/Example_1_OpenET_FlowMeter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! git clone https://github.com/sciencebyAJ/OWRD_Training_20240927.git

Cloning into 'OWRD_Training_20240927'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (96/96), done.
remote: Total 103 (delta 36), reused 26 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (103/103), 805.65 KiB | 3.50 MiB/s, done.
Resolving deltas: 100% (36/36), done.


In [3]:
import geopandas as gpd
import glob
import matplotlib.pyplot as plt


In [4]:
fields = gpd.read_file('/content/OWRD_Training_20240927/Shapefiles/flowmeter_shapefiles_v3/flowmeter_subset_fields_ORid_wgs84.shp')
fields_ss = fields[['OPENET_ID','Acres','SWU_ID','geometry']]

In [5]:
fields_ss.head()

,OPENET_ID,Acres,SWU_ID,geometry
0,ORx_24912,2.563825,SWU_UMAT0008,"POLYGON ((-118.44442 45.97865, -118.44443 45.9..."
1,ORx_24915,9.323391,SWU_UMAT0008,"POLYGON ((-118.44258 45.97911, -118.44187 45.9..."
2,ORx_25179,0.960043,SWU_UMAT0013,"POLYGON ((-118.38199 45.95786, -118.38197 45.9..."
3,ORx_25334,4.966444,SWU_UMAT0013,"POLYGON ((-118.38199 45.95744, -118.38187 45.9..."
4,ORx_25338,4.256518,SWU_UMAT0013,"POLYGON ((-118.38315 45.95842, -118.38204 45.9..."


In [6]:
fields_ss.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
hb = gpd.read_file('/content/OWRD_Training_20240927/Shapefiles/malheur_lake_basin_wgs84z11/malheur_lake_basin_wgs84z11.shp')

In [9]:
import ee
ee.Authenticate()
gcpid = 'csumb-et-tools' #@param{type: 'string'}
ee.Initialize(project= gcpid)
import geemap


In [10]:
# fig,axs = plt.subplots()

fields_utm = fields_ss.to_crs(epsg=32610)
# fields_utm.plot(ax=axs)
fields_utm['geometry'] = fields_utm.geometry.buffer(-20)
# fields_utm.plot(ax=axs, color='red')
fields_buf_wgs84 = fields_utm.to_crs(epsg=4326)

In [11]:
SIMScollection = (
    ee.ImageCollection("OpenET/SIMS/CONUS/GRIDMET/MONTHLY/v2_0")
    .filterDate("2023-05-01", "2023-10-01")
    )
SIMS = SIMScollection.select('et').sum()

EEMETRICcollection = (
    ee.ImageCollection("OpenET/EEMETRIC/CONUS/GRIDMET/MONTHLY/v2_0")
    .filterDate("2023-05-01", "2023-10-01")
    )
EEMETRIC = EEMETRICcollection.select('et').sum()


ENScollection = (
    ee.ImageCollection("OpenET/ENSEMBLE/CONUS/GRIDMET/MONTHLY/v2_0")
    .filterDate("2023-05-01", "2023-10-01")
    )
ENS = ENScollection.select('et_ensemble_mad').sum()
ENS_range = ENScollection.select('et_ensemble_max').sum().subtract(ENScollection.select('et_ensemble_min').sum())


vis = {
    "min": 0.0,
    "max": 900,
    "palette": [
    '9e6212', 'ac7d1d', 'ba9829', 'c8b434', 'd6cf40', 'bed44b', '9fcb51',
    '80c256', '61b95c', '42b062', '45b677', '49bc8d', '4dc2a2', '51c8b8',
    '55cece', '4db4ba', '459aa7', '3d8094', '356681', '2d4c6e',]
}
m = geemap.Map(basemap="Esri.WorldImagery")

m.add_gdf(fields_ss, layer_name="WW Fields")
m.add_gdf(fields_buf_wgs84,layer_name="WW Fields -20m buffer")
m.add_layer(SIMS, vis, "SIMS")
m.add_layer(EEMETRIC, vis, "EEMETRIC")
m.add_layer(ENS, vis, "ENS")
m.set_center(-118.424585, 45.970684, 14)

m


Map(center=[45.970684, -118.424585], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Se…

In [12]:
# Joe is going to get monthly time series of Ensemble ET and SIMS ET
# Joe is also goin to get monthly time series of gridmet precipitation for each field.